#### 1. 육류 가공식품 크롤링

In [8]:
# 라이브러리 로드
from selenium import webdriver #Selenium의 웹 드라이버를 사용하기 위한 모듈을 임포트
from selenium.webdriver.common.by import By #Selenium에서 사용하는 By 클래스를 임포트합. 웹 요소를 검색하는데 사용.
from selenium.webdriver.common.keys import Keys #키보드 입력 제어를 위해 Keys 클래스 임포트
from selenium.webdriver.chrome.service import Service #Chrome 드라이버 서비스를 사용하기 위한 모듈 임포트
from selenium.webdriver.chrome.options import Options #Chrome 드라이버 옵션을 설정하기 위한 클래스 임포트
from webdriver_manager.chrome import ChromeDriverManager #Chrome 드라이버를 자동으로 설치 및 관리하는데 사용되는 드라이버 매니저 임포트
from bs4 import BeautifulSoup #Beautiful soup 크롤링을위해
import pandas as pd
import requests #request 사용을 위해
import re #정규표현을  
import time #timesleep을 위해 사용
import numpy as np
import selenium
import pandas as pd
import os

In [25]:
#브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option('detach', True)
#불필요한 에러 메서지 없애기
chrome_options.add_experimental_option('excludeSwitches',['enable-logging'])
driver_path=ChromeDriverManager().install()
driver_path_tr=os.path.join(os.path.dirname(driver_path),'chromedriver.exe')
service = Service(executable_path = driver_path_tr)
url = "https://dart.fss.or.kr/dsae001/main.do"
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(5)

In [26]:
# 업종별 탭
upjong_btn=driver.find_element(By.CSS_SELECTOR,'#businessTab > a')
upjong_btn.click()

# 최상위 분류 
type_btn_1=driver.find_element(By.CSS_SELECTOR,'#root4547 > i')
type_btn_1.click()
time.sleep(1)

# 두 번째 상위 분류 
type_btn_2=driver.find_element(By.XPATH,'//*[@id="46"]/i')
type_btn_2.click()
time.sleep(1)

# 세 번째 상위 분류 
type_btn_3=driver.find_element(By.XPATH,'//*[@id="463"]/i')
type_btn_3.click()
time.sleep(1)

# 네 번재 상위 분류 
type_btn_4=driver.find_element(By.XPATH,'//*[@id="4632"]/i')
type_btn_4.click()
time.sleep(1)

# 마지막 분류
type_btn_end=driver.find_element(By.XPATH,'//*[@id="46321_anchor"]')
type_btn_end.click()
time.sleep(1)

In [27]:
# 빈 리스트 생성
table_list=[]

In [28]:
page_list=driver.find_elements(By.CSS_SELECTOR,'#listContents > div.psWrap > div.pageSkip > ul > li')
comp_list=driver.find_elements(By.XPATH,'//*[@id="corpTabel"]/tbody/tr')

In [30]:
for j in range(len(comp_list)):
    driver.find_element(By.XPATH,f'//*[@id="corpTabel"]/tbody/tr[{j+1}]/td[1]/span/a').click()
    time.sleep(1)
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    time.sleep(1)
    inform_table=soup.select('#corpDetailTabel > tbody')[0]
    table_list.append(inform_table)
    time.sleep(1)
comp_name=[]
owner_name=[]
address=[]
home_page=[]
phone_num=[]
fax_num=[]
comp_type=[]
start_date=[]
data_list=[]
for j in table_list:
    comp_name.append(j.select('tr>td')[2].text.strip())
    owner_name.append(j.select('tr>td')[4].text.split('\n')[0].lstrip())
    address.append(j.select('tr>td')[8].text.strip())
    home_page.append(j.select('tr>td')[9].text.strip())
    phone_num.append(j.select('tr>td')[11].text)
    fax_num.append(j.select('tr>td')[12].text)
    comp_type.append(j.select('tr>td')[13].text)
    start_date.append(pd.to_datetime(j.select('tr>td')[14].text,format='%Y-%m-%d'))
driver.close()

In [32]:
# 데이터프레임 생성

df_raw=pd.DataFrame({
    '회사명':comp_name,
    '대표자명':owner_name,
    '주소':address,
    '홈페이지':home_page,
    '전화번호':phone_num,
    '팩스번호':fax_num,
    '업종':comp_type,
    '설립일':start_date
    })

pork_done_df=df_raw
pork_done_df.head()


,회사명,대표자명,주소,홈페이지,전화번호,팩스번호,업종,설립일
0,가나안푸드,고안복,전라남도 나주시 산포면 매성리 291,,061-332-9998,-,육류 가공식품 도매업,2004-01-27
1,가우정푸드,오송균,경상남도 거창군 남상면 홍덕길 33 (대산리),,0559458100,0559458101,육류 가공식품 도매업,2014-07-16
2,농업회사법인강한돈,추연수,충청북도 충주시 주덕읍 충청대로 2312 (창전리),,043-854-5620,,육류 가공식품 도매업,2011-03-31
3,농업회사법인대광푸드,이병춘,충청북도 음성군 삼성면 대청로 54 (청용리),,043-535-1228,043-535-1227,육류 가공식품 도매업,2013-08-31
4,농업회사법인스마일팜,김천석,경기도 광주시 오포읍 봉골길134번길 57 (문형리) 스마일팜,,070-8877-6464,031-721-0777,육류 가공식품 도매업,2012-05-22


In [ ]:
# 육류 가공식품 데이터 저장
pork_done_df.to_csv('../data/육류 가공식품 도매기업.csv')

#### 2. 수산물 가공식품 크롤링

In [44]:
# 드라이버 접속
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(3)

# 업종별 탭
upjong_btn=driver.find_element(By.CSS_SELECTOR,'#businessTab > a')
upjong_btn.click()

# 최상위 분류 
type_btn_1=driver.find_element(By.CSS_SELECTOR,'#root4547 > i')
type_btn_1.click()
time.sleep(1)

# 두 번째 상위 분류 
type_btn_2=driver.find_element(By.XPATH,'//*[@id="46"]/i')
type_btn_2.click()
time.sleep(1)

# 세 번째 상위 분류 
type_btn_3=driver.find_element(By.XPATH,'//*[@id="463"]/i')
type_btn_3.click()
time.sleep(1)

# 네 번재 상위 분류 
type_btn_4=driver.find_element(By.XPATH,'//*[@id="4632"]/i')
type_btn_4.click()
time.sleep(1)

# 마지막 분류
type_btn_end=driver.find_element(By.XPATH,'//*[@id="46322_anchor"]')
type_btn_end.click()
time.sleep(1)

In [40]:
# 빈 리스트 생성
table_list=[]

In [41]:
page_list=driver.find_elements(By.CSS_SELECTOR,'#listContents > div.psWrap > div.pageSkip > ul > li')
comp_list=driver.find_elements(By.XPATH,'//*[@id="corpTabel"]/tbody/tr')

In [42]:
for j in range(len(comp_list)):
    driver.find_element(By.XPATH,f'//*[@id="corpTabel"]/tbody/tr[{j+1}]/td[1]/span/a').click()
    time.sleep(1)
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    time.sleep(1)
    inform_table=soup.select('#corpDetailTabel > tbody')[0]
    table_list.append(inform_table)
    time.sleep(1)
comp_name=[]
owner_name=[]
address=[]
home_page=[]
phone_num=[]
fax_num=[]
comp_type=[]
start_date=[]
data_list=[]
for j in table_list:
    comp_name.append(j.select('tr>td')[2].text.strip())
    owner_name.append(j.select('tr>td')[4].text.split('\n')[0].lstrip())
    address.append(j.select('tr>td')[8].text.strip())
    home_page.append(j.select('tr>td')[9].text.strip())
    phone_num.append(j.select('tr>td')[11].text)
    fax_num.append(j.select('tr>td')[12].text)
    comp_type.append(j.select('tr>td')[13].text)
    start_date.append(pd.to_datetime(j.select('tr>td')[14].text,format='%Y-%m-%d'))
driver.close()

In [45]:
# 데이터프레임 생성

df_raw=pd.DataFrame({
    '회사명':comp_name,
    '대표자명':owner_name,
    '주소':address,
    '홈페이지':home_page,
    '전화번호':phone_num,
    '팩스번호':fax_num,
    '업종':comp_type,
    '설립일':start_date
    })

sf_df = df_raw
sf_df.head()

,회사명,대표자명,주소,홈페이지,전화번호,팩스번호,업종,설립일
0,곰소식품,이팔규,서울특별시 동대문구 경동시장로 26 (제기동),,029665206,,수산물 가공식품 도매업,2014-04-20
1,금성,이용완,"부산광역시 서구 원양로 1 비동 608호(암남동, 수산가공선진화단지)",,051-808-8876,070-8233-2711,수산물 가공식품 도매업,2005-03-22
2,대평물산,홍종순,"부산광역시 부산진구 부전로170번길 21 (부전동 ,2층)",,07073920223,0517026753,수산물 가공식품 도매업,2010-03-30
3,모노링크,허태영,"서울특별시 송파구 삼전로2길 51 (삼전동, 일본식품아울렛)",www.monolink.kr,02-425-6837,02-425-6838,수산물 가공식품 도매업,2000-07-10
4,부산세광식품,김수진,부산광역시 강서구 녹산산업중로 381 (송정동),,051-264-4324,051-262-9008,수산물 가공식품 도매업,1972-01-26


In [ ]:
# 육류 가공식품 데이터 저장
sf_df.to_csv('../data/수산물 가공식품 도매기업.csv')

#### 3. 빵류, 과자류, 당류, 초콜릿 도매업 크롤링

In [46]:
# 드라이버 접속
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(3)

# 업종별 탭
upjong_btn=driver.find_element(By.CSS_SELECTOR,'#businessTab > a')
upjong_btn.click()

# 최상위 분류 
type_btn_1=driver.find_element(By.CSS_SELECTOR,'#root4547 > i')
type_btn_1.click()
time.sleep(1)

# 두 번째 상위 분류 
type_btn_2=driver.find_element(By.XPATH,'//*[@id="46"]/i')
type_btn_2.click()
time.sleep(1)

# 세 번째 상위 분류 
type_btn_3=driver.find_element(By.XPATH,'//*[@id="463"]/i')
type_btn_3.click()
time.sleep(1)

# 네 번재 상위 분류 
type_btn_4=driver.find_element(By.XPATH,'//*[@id="4632"]/i')
type_btn_4.click()
time.sleep(1)

# 마지막 분류
type_btn_end=driver.find_element(By.XPATH,'//*[@id="46323_anchor"]')
type_btn_end.click()
time.sleep(1)

In [47]:
# 빈 리스트 생성
table_list=[]

In [48]:
page_list=driver.find_elements(By.CSS_SELECTOR,'#listContents > div.psWrap > div.pageSkip > ul > li')
comp_list=driver.find_elements(By.XPATH,'//*[@id="corpTabel"]/tbody/tr')

In [49]:
for j in range(len(comp_list)):
    driver.find_element(By.XPATH,f'//*[@id="corpTabel"]/tbody/tr[{j+1}]/td[1]/span/a').click()
    time.sleep(1)
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    time.sleep(1)
    inform_table=soup.select('#corpDetailTabel > tbody')[0]
    table_list.append(inform_table)
    time.sleep(1)
comp_name=[]
owner_name=[]
address=[]
home_page=[]
phone_num=[]
fax_num=[]
comp_type=[]
start_date=[]
data_list=[]
for j in table_list:
    comp_name.append(j.select('tr>td')[2].text.strip())
    owner_name.append(j.select('tr>td')[4].text.split('\n')[0].lstrip())
    address.append(j.select('tr>td')[8].text.strip())
    home_page.append(j.select('tr>td')[9].text.strip())
    phone_num.append(j.select('tr>td')[11].text)
    fax_num.append(j.select('tr>td')[12].text)
    comp_type.append(j.select('tr>td')[13].text)
    start_date.append(pd.to_datetime(j.select('tr>td')[14].text,format='%Y-%m-%d'))
driver.close()

In [50]:
# 데이터프레임 생성
df_raw=pd.DataFrame({
    '회사명':comp_name,
    '대표자명':owner_name,
    '주소':address,
    '홈페이지':home_page,
    '전화번호':phone_num,
    '팩스번호':fax_num,
    '업종':comp_type,
    '설립일':start_date
    })

bread_df = df_raw
bread_df.head()

,회사명,대표자명,주소,홈페이지,전화번호,팩스번호,업종,설립일
0,그라쉐,김영준,충청북도 진천군 이월면 사곡리 615-7,,031-715-7780,031-715-7709,"빵류, 과자류, 당류, 초콜릿 도매업",2005-05-03
1,글로벌하베스트,"GAO LEI, DONG WENXIANG","인천광역시 서구 청라한내로72번길 7-25 (청라동, 리버사이드크루즈몰) 409호",,032-563-0908,032-563-0903,"빵류, 과자류, 당류, 초콜릿 도매업",2023-06-02
2,길무역,한동민,경기도 성남시 수정구 복정동 30,,02-418-1211,02-418-1352,"빵류, 과자류, 당류, 초콜릿 도매업",1985-06-25
3,농업회사법인오리온농협,박민규,경상남도 밀양시 부북면 점필재로 229-1 (제대리),,055-802-5000,055-802-8034,"빵류, 과자류, 당류, 초콜릿 도매업",2016-09-27
4,떡보의하루,성우진,대구광역시 달서구 이곡동로 11 1층(이곡동),,053-593-4427,053-592-4427,"빵류, 과자류, 당류, 초콜릿 도매업",2010-03-16


In [ ]:
# 빵류, 과자류 가공식품 데이터 저장
bread_df.to_csv('../data/빵류, 과자류 도매기업.csv')

#### 4. 낙농품 및 동식물성 유지 도매업 크롤링

In [51]:
# 드라이버 접속
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(3)

# 업종별 탭
upjong_btn=driver.find_element(By.CSS_SELECTOR,'#businessTab > a')
upjong_btn.click()

# 최상위 분류 
type_btn_1=driver.find_element(By.CSS_SELECTOR,'#root4547 > i')
type_btn_1.click()
time.sleep(1)

# 두 번째 상위 분류 
type_btn_2=driver.find_element(By.XPATH,'//*[@id="46"]/i')
type_btn_2.click()
time.sleep(1)

# 세 번째 상위 분류 
type_btn_3=driver.find_element(By.XPATH,'//*[@id="463"]/i')
type_btn_3.click()
time.sleep(1)

# 네 번재 상위 분류 
type_btn_4=driver.find_element(By.XPATH,'//*[@id="4632"]/i')
type_btn_4.click()
time.sleep(1)

# 마지막 분류
type_btn_end=driver.find_element(By.XPATH,'//*[@id="46324_anchor"]')
type_btn_end.click()
time.sleep(1)

In [52]:
# 빈 리스트 생성
table_list=[]

In [53]:
page_list=driver.find_elements(By.CSS_SELECTOR,'#listContents > div.psWrap > div.pageSkip > ul > li')
comp_list=driver.find_elements(By.XPATH,'//*[@id="corpTabel"]/tbody/tr')

In [54]:
for j in range(len(comp_list)):
    driver.find_element(By.XPATH,f'//*[@id="corpTabel"]/tbody/tr[{j+1}]/td[1]/span/a').click()
    time.sleep(1)
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    time.sleep(1)
    inform_table=soup.select('#corpDetailTabel > tbody')[0]
    table_list.append(inform_table)
    time.sleep(1)
comp_name=[]
owner_name=[]
address=[]
home_page=[]
phone_num=[]
fax_num=[]
comp_type=[]
start_date=[]
data_list=[]
for j in table_list:
    comp_name.append(j.select('tr>td')[2].text.strip())
    owner_name.append(j.select('tr>td')[4].text.split('\n')[0].lstrip())
    address.append(j.select('tr>td')[8].text.strip())
    home_page.append(j.select('tr>td')[9].text.strip())
    phone_num.append(j.select('tr>td')[11].text)
    fax_num.append(j.select('tr>td')[12].text)
    comp_type.append(j.select('tr>td')[13].text)
    start_date.append(pd.to_datetime(j.select('tr>td')[14].text,format='%Y-%m-%d'))
driver.close()

In [55]:
# 데이터프레임 생성

df_raw=pd.DataFrame({
    '회사명':comp_name,
    '대표자명':owner_name,
    '주소':address,
    '홈페이지':home_page,
    '전화번호':phone_num,
    '팩스번호':fax_num,
    '업종':comp_type,
    '설립일':start_date
    })

milk_df = df_raw
milk_df.head()

,회사명,대표자명,주소,홈페이지,전화번호,팩스번호,업종,설립일
0,경일푸드,김성희,경기도 시흥시 군자로10번길 14-2 (월곶동),,031-312-6262,031-313-6262,낙농품 및 동ㆍ식물성 유지 도매업,2016-08-01
1,로브,주성완,"서울특별시 종로구 사직로8길 34 831호 (내수동, 경희궁의아침3단지)",,02-552-0804,,낙농품 및 동ㆍ식물성 유지 도매업,2007-04-05
2,롯데나뚜루,이은학,서울특별시 영등포구 양평동5가 21번지,www.natuur.co.kr,02-2670-6433,02-2670-6868,낙농품 및 동ㆍ식물성 유지 도매업,2011-10-01
3,범호통상,윤홍묵,경기도 용인시 처인구 포곡읍 한금로 153 (금어리),,031-702-5274,031-706-5274,낙농품 및 동ㆍ식물성 유지 도매업,2001-10-25
4,베스트로,부종일,"경기도 과천시 관문로 166 (중앙동, 주공아파트) b12호",www.ibestro.com/,02-502-0447,02-502-0448,낙농품 및 동ㆍ식물성 유지 도매업,2005-12-12


In [ ]:
# 낙농품 및 동식물성 유지 도매업 데이터 저장
milk_df.to_csv('../data/낙농품 및 동식물성 유지 도매기업.csv')

#### 5. 커피 및 차류 도매업 크롤링

In [56]:
# 드라이버 접속
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(3)

# 업종별 탭
upjong_btn=driver.find_element(By.CSS_SELECTOR,'#businessTab > a')
upjong_btn.click()

# 최상위 분류 
type_btn_1=driver.find_element(By.CSS_SELECTOR,'#root4547 > i')
type_btn_1.click()
time.sleep(1)

# 두 번째 상위 분류 
type_btn_2=driver.find_element(By.XPATH,'//*[@id="46"]/i')
type_btn_2.click()
time.sleep(1)

# 세 번째 상위 분류 
type_btn_3=driver.find_element(By.XPATH,'//*[@id="463"]/i')
type_btn_3.click()
time.sleep(1)

# 네 번재 상위 분류 
type_btn_4=driver.find_element(By.XPATH,'//*[@id="4632"]/i')
type_btn_4.click()
time.sleep(1)

# 마지막 분류
type_btn_end=driver.find_element(By.XPATH,'//*[@id="46325_anchor"]')
type_btn_end.click()
time.sleep(1)

In [57]:
# 빈 리스트 생성
table_list=[]

In [58]:
page_list=driver.find_elements(By.CSS_SELECTOR,'#listContents > div.psWrap > div.pageSkip > ul > li')
comp_list=driver.find_elements(By.XPATH,'//*[@id="corpTabel"]/tbody/tr')

In [59]:
for j in range(len(comp_list)):
    driver.find_element(By.XPATH,f'//*[@id="corpTabel"]/tbody/tr[{j+1}]/td[1]/span/a').click()
    time.sleep(1)
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    time.sleep(1)
    inform_table=soup.select('#corpDetailTabel > tbody')[0]
    table_list.append(inform_table)
    time.sleep(1)
comp_name=[]
owner_name=[]
address=[]
home_page=[]
phone_num=[]
fax_num=[]
comp_type=[]
start_date=[]
data_list=[]
for j in table_list:
    comp_name.append(j.select('tr>td')[2].text.strip())
    owner_name.append(j.select('tr>td')[4].text.split('\n')[0].lstrip())
    address.append(j.select('tr>td')[8].text.strip())
    home_page.append(j.select('tr>td')[9].text.strip())
    phone_num.append(j.select('tr>td')[11].text)
    fax_num.append(j.select('tr>td')[12].text)
    comp_type.append(j.select('tr>td')[13].text)
    start_date.append(pd.to_datetime(j.select('tr>td')[14].text,format='%Y-%m-%d'))
driver.close()

In [60]:
# 데이터프레임 생성

df_raw=pd.DataFrame({
    '회사명':comp_name,
    '대표자명':owner_name,
    '주소':address,
    '홈페이지':home_page,
    '전화번호':phone_num,
    '팩스번호':fax_num,
    '업종':comp_type,
    '설립일':start_date
    })

coffee_df = df_raw
coffee_df.head()

,회사명,대표자명,주소,홈페이지,전화번호,팩스번호,업종,설립일
0,공정한착한커피,김신아,서울특별시 강남구 영동대로 602 6층 E43호(삼성동),,02-6207-7111,,커피 및 차류 도매업,2020-07-10
1,구일종합식품,김진완,"서울특별시 강동구 양재대로121길 7 (천호동, 거송빌딩) 301호",,024705319,024715319,커피 및 차류 도매업,2004-07-30
2,더젬코리아,김미희,부산광역시 해운대구 송정2로 27 더젬코리아,,051-583-5498,051-584-5414,커피 및 차류 도매업,2016-10-07
3,뮤제오,서동의,"서울특별시 강서구 양천로 583 (염창동, 우림블루나인비즈니스센터) A동 609호",www.caffemuseo.co.kr,02-2607-0918,02-2607-0129,커피 및 차류 도매업,2006-04-21
4,미래에프앤지,하광,경기도 김포시 고촌읍 금포로 653 (향산리),,15440977,,커피 및 차류 도매업,2010-08-01


In [ ]:
# 낙농품 및 동식물성 유지 도매업 데이터 저장
coffee_df.to_csv('../data/낙농품 및 동식물성 유지 도매기업.csv')

#### 6. 조미료 도매업 크롤링

In [61]:
# 드라이버 접속
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(3)

# 업종별 탭
upjong_btn=driver.find_element(By.CSS_SELECTOR,'#businessTab > a')
upjong_btn.click()

# 최상위 분류 
type_btn_1=driver.find_element(By.CSS_SELECTOR,'#root4547 > i')
type_btn_1.click()
time.sleep(1)

# 두 번째 상위 분류 
type_btn_2=driver.find_element(By.XPATH,'//*[@id="46"]/i')
type_btn_2.click()
time.sleep(1)

# 세 번째 상위 분류 
type_btn_3=driver.find_element(By.XPATH,'//*[@id="463"]/i')
type_btn_3.click()
time.sleep(1)

# 네 번재 상위 분류 
type_btn_4=driver.find_element(By.XPATH,'//*[@id="4632"]/i')
type_btn_4.click()
time.sleep(1)

# 마지막 분류
type_btn_end=driver.find_element(By.XPATH,'//*[@id="46326_anchor"]')
type_btn_end.click()
time.sleep(1)

In [62]:
# 빈 리스트 생성
table_list=[]

In [63]:
page_list=driver.find_elements(By.CSS_SELECTOR,'#listContents > div.psWrap > div.pageSkip > ul > li')
comp_list=driver.find_elements(By.XPATH,'//*[@id="corpTabel"]/tbody/tr')

In [64]:
for j in range(len(comp_list)):
    driver.find_element(By.XPATH,f'//*[@id="corpTabel"]/tbody/tr[{j+1}]/td[1]/span/a').click()
    time.sleep(1)
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    time.sleep(1)
    inform_table=soup.select('#corpDetailTabel > tbody')[0]
    table_list.append(inform_table)
    time.sleep(1)
comp_name=[]
owner_name=[]
address=[]
home_page=[]
phone_num=[]
fax_num=[]
comp_type=[]
start_date=[]
data_list=[]
for j in table_list:
    comp_name.append(j.select('tr>td')[2].text.strip())
    owner_name.append(j.select('tr>td')[4].text.split('\n')[0].lstrip())
    address.append(j.select('tr>td')[8].text.strip())
    home_page.append(j.select('tr>td')[9].text.strip())
    phone_num.append(j.select('tr>td')[11].text)
    fax_num.append(j.select('tr>td')[12].text)
    comp_type.append(j.select('tr>td')[13].text)
    start_date.append(pd.to_datetime(j.select('tr>td')[14].text,format='%Y-%m-%d'))
driver.close()

In [65]:
# 데이터프레임 생성

df_raw=pd.DataFrame({
    '회사명':comp_name,
    '대표자명':owner_name,
    '주소':address,
    '홈페이지':home_page,
    '전화번호':phone_num,
    '팩스번호':fax_num,
    '업종':comp_type,
    '설립일':start_date
    })

con_df = df_raw
con_df.head()

,회사명,대표자명,주소,홈페이지,전화번호,팩스번호,업종,설립일
0,더쉐프,김태경,부산광역시 연제구 좌수영로 290 (연산동),www.tenpercentcoffee.com,051-995-7779,051-996-7779,조미료 도매업,2013-06-18
1,예광인터내셔널,안철훈,"경기도 하남시 대청로 15 1512호(신장동, 오피스텔트레벨)",http:www.yekwang.kr,031-793-5370,031-793-5470,조미료 도매업,2000-12-12
2,우리상사,구기운,충청남도 계룡시 두마면 왕대로 69-14 (왕대리),,042-824-1614,042-824-1618,조미료 도매업,2009-02-17
3,제이씨월드,윤순옥,서울특별시 관악구 봉천로 561 (봉천동) 삼정빌딩 6층,,02-881-0662,02-881-0699,조미료 도매업,2019-07-01
4,지보단코리아,강현구,"서울특별시 서초구 마방로 60 (양재동, 동원에프앤비빌딩) 11,12층",https://www.givaudan.com/,02-2149-5800,02-2149-5805,조미료 도매업,1994-05-01


In [ ]:
# 조미료 도매업 데이터 저장
con_df.to_csv('../data/조미료 도매기업.csv')

#### 7. 기타 가공식품 도매업 크롤링

In [87]:
# 드라이버 접속
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(3)

# 업종별 탭
upjong_btn=driver.find_element(By.CSS_SELECTOR,'#businessTab > a')
upjong_btn.click()

# 최상위 분류 
type_btn_1=driver.find_element(By.CSS_SELECTOR,'#root4547 > i')
type_btn_1.click()
time.sleep(1)

# 두 번째 상위 분류 
type_btn_2=driver.find_element(By.XPATH,'//*[@id="46"]/i')
type_btn_2.click()
time.sleep(1)

# 세 번째 상위 분류 
type_btn_3=driver.find_element(By.XPATH,'//*[@id="463"]/i')
type_btn_3.click()
time.sleep(1)

# 네 번재 상위 분류 
type_btn_4=driver.find_element(By.XPATH,'//*[@id="4632"]/i')
type_btn_4.click()
time.sleep(1)

# 마지막 분류
type_btn_end=driver.find_element(By.XPATH,'//*[@id="46329_anchor"]')
type_btn_end.click()
time.sleep(1)

In [88]:
# 빈 리스트 생성
table_list=[]

In [89]:
page_list=driver.find_elements(By.CSS_SELECTOR,'#listContents > div.psWrap > div.pageSkip > ul > li')

In [90]:
for i in page_list:
    if page_list.index(i)!=0:
        i.click()
        time.sleep(1)
    comp_list=driver.find_elements(By.XPATH,'//*[@id="corpTabel"]/tbody/tr')
    for j in range(len(comp_list)):
        driver.find_element(By.XPATH,f'//*[@id="corpTabel"]/tbody/tr[{j+1}]/td[1]/span/a').click()
        time.sleep(1)
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        time.sleep(1)
        inform_table=soup.select('#corpDetailTabel > tbody')[0]
        table_list.append(inform_table)
        time.sleep(1)

In [91]:
# 빈 리스트 생성
comp_name=[]
owner_name=[]
address=[]
home_page=[]
phone_num=[]
fax_num=[]
comp_type=[]
start_date=[]
ata_list=[]

In [92]:
# 데이터프레임 생성
for j in table_list:
    comp_name.append(j.select('tr>td')[2].text.strip())
    owner_name.append(j.select('tr>td')[4].text.split('\n')[0].lstrip())
    address.append(j.select('tr>td')[8].text.strip())
    home_page.append(j.select('tr>td')[9].text.strip())
    phone_num.append(j.select('tr>td')[11].text)
    fax_num.append(j.select('tr>td')[12].text)
    comp_type.append(j.select('tr>td')[13].text)
    start_date.append(pd.to_datetime(j.select('tr>td')[14].text,format='%Y-%m-%d'))

In [93]:
df_raw=pd.DataFrame({
    '회사명':comp_name,
    '대표자명':owner_name,
    '주소':address,
    '홈페이지':home_page,
    '전화번호':phone_num,
    '팩스번호':fax_num,
    '업종':comp_type,
    '설립일':start_date
    })

etc_df=df_raw
etc_df

,회사명,대표자명,주소,홈페이지,전화번호,팩스번호,업종,설립일
0,교원더오름,장평순,"서울특별시 중구 을지로 51 (을지로2가, 내외빌딩)",,02-397-2916,,기타 가공식품 도매업,2018-02-15
1,그린스토어,박영창,"경기도 성남시 분당구 느티로 16 (정자동, 젤존타워1)",www.green-store.co.kr,0317787844,0317187032,기타 가공식품 도매업,2011-03-29
2,그린알로에,정광숙,"광주광역시 동구 독립로 282-2 (대인동),1층",www.greenaloe.co.kr,062-223-6589,062-222-1344,기타 가공식품 도매업,1997-07-01
3,글루업,박경준,서울특별시 강남구 압구정로30길 67 (신사동),,07049035750,,기타 가공식품 도매업,2018-09-21
4,나팔꽃에프앤비,김영옥 정명호,전라남도 신안군 안좌면 산두리길 86 (대리),napal.co.kr/,010-8624-5433,02-518-5455,기타 가공식품 도매업,2018-10-19
...,...,...,...,...,...,...,...,...
66,한국마그나스,안영옥,"경기도 남양주시 늘을2로14번길 5 (호평동, 샤르망메디칼프라자) 7층 702호",,070-4907-3460,031-592-4659,기타 가공식품 도매업,1991-09-05
67,한국바이오,지정근,경기도 평택시 서탄면 발안로 1198-7 (수월암리),,07047131801,,기타 가공식품 도매업,2021-09-14
68,한국씨엔에스,승준호,인천광역시 서구 봉수대로 1274 (왕길동),,032-888-0077,,기타 가공식품 도매업,2015-08-01
69,헬스하우스,김우형,서울특별시 동대문구 장한로 79 (장안동),health38401.cafe24.com/,02-926-3840,02-929-3840,기타 가공식품 도매업,2008-10-29


In [ ]:
etc_df.to_csv('../data/기타 도매기업.csv')